In [2]:
# 💻 Complete Happy Face Detector Code for Google Colab (One Shell)

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode
import cv2
from google.colab.patches import cv2_imshow # Specific for Colab
import os

# --- Configuration and Mock Data Setup ---
emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
num_classes = len(emotion_labels)
img_rows, img_cols = 48, 48
input_shape = (img_rows, img_cols, 1)
model_filename = "happy_face_detector_model.h5"
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# --- 1. Model Definition and Training (for a runnable example) ---

def create_model(input_shape):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))

    return model

# Generate Mock Data
print("🛠️ Generating and training model with mock data for demonstration purposes...")
X_mock = np.random.rand(100, img_rows, img_cols, 1).astype('float32') / 255.0
y_mock = np.random.randint(0, num_classes, 100)
y_mock = to_categorical(y_mock, num_classes=num_classes)
X_train, X_test, y_train, y_test = train_test_split(X_mock, y_mock, test_size=0.2, random_state=42)

# Train and Save
model = create_model(input_shape)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=32, epochs=2, verbose=0) # Minimal training
model.save(model_filename)
print("✅ Model trained and saved.")


# --- 2. Camera Utility Function (JavaScript/Python integration) ---

def take_photo(filename='photo.jpg', quality=0.8):
    js = Javascript('''
        async function takePhoto(quality) {
          const div = document.createElement('div');
          const video = document.createElement('video');
          video.style.display = 'block';
          const stream = await navigator.mediaDevices.getUserMedia({video: true});

          document.body.appendChild(div);
          div.appendChild(video);
          video.srcObject = stream;
          await video.play();

          // Wait for the video to load and then set the canvas size
          await new Promise(resolve => video.onloadedmetadata = resolve);

          const canvas = document.createElement('canvas');
          canvas.width = video.videoWidth;
          canvas.height = video.videoHeight;
          const context = canvas.getContext('2d');

          // Add a button to capture the image
          const button = document.createElement('button');
          button.textContent = 'Capture';
          div.appendChild(button);

          await new Promise(resolve => button.onclick = resolve);

          context.drawImage(video, 0, 0);
          stream.getVideoTracks()[0].stop();
          div.remove();
          return canvas.toDataURL('image/jpeg', quality);
        }
        ''')
    display(js)
    data = eval_js('takePhoto({})'.format(quality))
    binary = b64decode(data.split(',')[1])
    with open(filename, 'wb') as f:
        f.write(binary)
    return filename

# --- 3. Preprocessing and Prediction Functions ---

def preprocess_face(face_img):
    """Resizes and normalizes the detected face for the model."""
    gray_face = cv2.cvtColor(face_img, cv2.COLOR_BGR2GRAY)
    resized_face = cv2.resize(gray_face, (48, 48), interpolation=cv2.INTER_AREA)
    # Normalize and reshape for the model (48x48x1)
    normalized_face = resized_face.astype('float32') / 255.0
    input_face = np.expand_dims(normalized_face, axis=0) # Add batch dimension
    input_face = np.expand_dims(input_face, axis=-1) # Add channel dimension
    return input_face

def predict_emotion_from_photo(filename='captured_face.jpg'):
    """Captures photo, detects face, and predicts emotion."""
    try:
        model = load_model(model_filename)
    except Exception as e:
        print(f"FATAL ERROR: Could not load model '{model_filename}'. Ensure training was successful. Error: {e}")
        return

    print("\n📸 **Action Required:** Camera opening. Please click 'Capture' when ready! 📸")

    # Capture the photo
    try:
        image_file = take_photo(filename)
        print(f'Saved photo to {image_file}')
    except Exception as err:
        print(f"Camera capture failed: {str(err)}")
        return

    # Read and process the image
    img = cv2.imread(image_file)
    if img is None:
        print(f"Error: Could not read image file {image_file}")
        return

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    if len(faces) == 0:
        text = "No face detected in the photo."
        cv2.putText(img, text, (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        print(f"⚠️ {text}")
    else:
        for (x, y, w, h) in faces:
            face_img = img[y:y+h, x:x+w]
            input_face = preprocess_face(face_img)

            # Predict emotion
            predictions = model.predict(input_face, verbose=0)
            predicted_class_index = np.argmax(predictions[0])
            predicted_emotion = emotion_labels[predicted_class_index]
            confidence = predictions[0][predicted_class_index] * 100

            # Draw results on the image
            color = (0, 255, 0) if predicted_emotion == 'Happy' else (0, 0, 255)
            label = f"{predicted_emotion}: {confidence:.2f}%"
            cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
            cv2.putText(img, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)

            # Output Result
            if predicted_emotion == 'Happy':
                print(f"🥳 **PREDICTION:** You look **{predicted_emotion}** with **{confidence:.2f}%** confidence!")
            else:
                print(f"🤔 **PREDICTION:** The detected emotion is **{predicted_emotion}** ({confidence:.2f}%).")

    # Display the final image with results
    print("\n🖼️ Displaying result:")
    cv2_imshow(img)

# --- 4. EXECUTE THE DETECTION PROCESS ---
predict_emotion_from_photo()

Camera capture failed: 'NoneType' object has no attribute 'split'
